In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/churn-predictions-personal/Churn_Predictions.csv


In [2]:
!pip install tensorflow==2.15.0 scikit-learn tensorboard matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


1. **`StandardScaler`**: This tool is used to standardize features by removing the mean and scaling to unit variance. In other words, it transforms your data so that it has a mean of 0 and a standard deviation of 1. This is useful when your features have different scales and you want to ensure that they are treated equally in a machine learning model.

   - Example: If you have a dataset with features like height (in centimeters) and weight (in kilograms), their scales are very different. `StandardScaler` adjusts them so they both have a similar range, making the model more stable and accurate.

2. **`LabelEncoder`**: This tool is used to convert categorical labels (like "cat", "dog", "mouse") into a numerical form (like 0, 1, 2). It's useful when your machine learning model only accepts numerical input and you need to encode categorical variables.

   - Example: If you have a column in your dataset with the labels "red", "blue", and "green", `LabelEncoder` will convert these to 0, 1, and 2 respectively.

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [6]:
# Load the Dataset

data = pd.read_csv("/kaggle/input/churn-predictions-personal/Churn_Predictions.csv")
data.head()                   

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [9]:
data.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [12]:
data.shape

(10000, 14)

# Feature Transformation

## Drop irrelevant columns

In [16]:
# Preprocessing 
# Drop irrelevant columns
data = data.drop(["RowNumber", "CustomerId", "Surname"], axis = 1)

In [17]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Encoding

In [20]:
le = LabelEncoder()
data["Gender"] = le.fit_transform(data["Gender"])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [22]:
data["Gender"].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

We use one-hot encoding for the **geography** column to convert categorical data into a numeric format that machine learning models can use. 

### Key Points:
1. **Avoids Misinterpretation:** 
   - It prevents the model from assuming any order or importance among categories (like France, Spain, Germany).

2. **Creates Binary Features:**
   - It turns each category into a separate column with binary values (0 or 1), making it easy for the model to process.

3. **Prevents Bias:**
   - Ensures that no category is given more weight just because of its numerical value.

So, one-hot encoding transforms categorical data into a suitable numeric format without implying any order or relationship between the categories.

In [21]:
data["Geography"].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [26]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
geo_encode = ohe.fit_transform(data[["Geography"]])

In [27]:
geo_encode

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [28]:
geo_encode.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [31]:
ohe.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [32]:
geo_encoded_df = pd.DataFrame(geo_encode.toarray(), columns = ohe.get_feature_names_out(["Geography"]))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [35]:
# Combine all columns
data = pd.concat([data.drop("Geography", axis = 1), geo_encoded_df], axis = 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Gender             10000 non-null  int64  
 2   Age                10000 non-null  int64  
 3   Tenure             10000 non-null  int64  
 4   Balance            10000 non-null  float64
 5   NumOfProducts      10000 non-null  int64  
 6   HasCrCard          10000 non-null  int64  
 7   IsActiveMember     10000 non-null  int64  
 8   EstimatedSalary    10000 non-null  float64
 9   Exited             10000 non-null  int64  
 10  Geography_France   10000 non-null  float64
 11  Geography_Germany  10000 non-null  float64
 12  Geography_Spain    10000 non-null  float64
dtypes: float64(5), int64(8)
memory usage: 1015.8 KB


## Saving the Encoder
It's important to maintain consistency between the training and prediction phases, especially when dealing with categorical data. During training, LabelEncoder and OHE learn how to convert categories into numerical values. If this mapping isn't preserved, the model might receive different encodings at inference time, leading to errors. By saving these encoders with Pickle, we ensure that the exact same transformations are applied when the model is used in production, thereby maintaining consistency and accuracy

In [37]:
with open("label_encoder_gender.pkl", "wb") as file:
    pickle.dump(le, file)

In [38]:
with open("ohe_geo.pkl", "wb") as file:
    pickle.dump(ohe, file)

In [39]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


## Splitting the Dataset and Scaling Features

In [44]:
# Split the dataset

X = data.drop(["Exited"], axis = 1)
y= data["Exited"]

# Split in Train Test Split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Scale the data
scaler= StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [47]:
X_train.shape

(8000, 12)

In [48]:
y_train.shape

(8000,)

## Saving the Scaler

In [49]:
with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

# Artificial Neural Network (ANN)

An Artificial Neural Network (ANN) is a computational model inspired by the way biological neural networks in the human brain process information. It is a key component of deep learning, a subset of machine learning. ANNs consist of interconnected layers of nodes (or "neurons"), which process input data and learn patterns through training.

#### Structure:
- **Input Layer**: Receives the initial data.
- **Hidden Layers**: Intermediate layers where computations occur. These layers learn features from the data by adjusting weights through a process called backpropagation.
- **Output Layer**: Produces the final prediction or classification.

#### Key Concepts:
- **Weights and Biases**: Parameters that the network adjusts during training to minimize error.
- **Activation Functions**: Functions like ReLU, Sigmoid, or Tanh applied to each neuron’s output to introduce non-linearity, enabling the network to learn complex patterns.
- **Learning Rate**: A hyperparameter that controls how much the model's weights are updated with respect to the loss gradient.

#### Applications:
ANNs are used in various fields such as image and speech recognition, natural language processing, and predictive analytics, offering powerful solutions to complex problems.

In summary, ANNs are the backbone of modern AI, enabling machines to learn from data and make intelligent decisions.

# ANN Implementation

In [53]:
# Import TensorFlow library for building and training neural networks
import tensorflow as tf

# Import Sequential model for creating linear stack of layers
from tensorflow.keras.models import Sequential

# Import Dense layer for adding fully connected layers to the model
from tensorflow.keras.layers import Dense

# Import EarlyStopping to halt training when a monitored metric stops improving
# Import TensorBoard to visualize training metrics and model performance
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

# Import datetime to manage and format timestamps for TensorBoard logs
import datetime

In [59]:
(X_train.shape[1],)

(12,)

## Build the model

In [60]:
# Build our Model
model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)), #Hidden Layer 1
    Dense(32, activation="relu"), #Hidden layer 2
    Dense(1, activation="sigmoid") # Output layer
]
)

In [61]:
model

In [63]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### **1. Optimizer**
- **Definition**: An optimizer is an algorithm used to adjust the weights of the neural network during training. Its goal is to minimize the loss function by iteratively updating the model parameters based on the gradients computed during backpropagation.
- **Purpose**: To find the optimal set of weights that results in the best performance of the model.
- **Common Types**:
  - **Gradient Descent**: Basic algorithm where weights are updated in the direction opposite to the gradient.
  - **Stochastic Gradient Descent (SGD)**: A variant where weights are updated more frequently using small batches of data.
  - **Adam (Adaptive Moment Estimation)**: Combines the advantages of two other extensions of SGD, adapting the learning rate for each parameter.

### **2. Loss Function**
- **Definition**: A loss function (or cost function) measures how well the model’s predictions match the actual target values. It quantifies the error between predicted outputs and true outputs.
- **Purpose**: To provide a feedback mechanism for the optimizer to improve the model's performance by minimizing this error.
- **Common Types**:
  - **Mean Squared Error (MSE)**: Used for regression tasks; measures the average squared difference between predicted and actual values.
  - **Binary Cross-Entropy**: Used for binary classification; measures the performance of a classification model whose output is a probability value between 0 and 1.
  - **Categorical Cross-Entropy**: Used for multi-class classification; measures the performance of a model where the output is a probability distribution across multiple classes.

### **3. Metric**
- **Definition**: Metrics are evaluation criteria used to monitor the performance of the model during training and testing. Unlike the loss function, metrics are not used to update the model weights but are used to assess the quality of the model's predictions.
- **Purpose**: To track the performance of the model and provide insights into how well it is learning.
- **Common Types**:
  - **Accuracy**: Measures the proportion of correct predictions out of all predictions made, used for classification tasks.
  - **Precision, Recall, F1-Score**: More detailed metrics for classification problems, especially useful in imbalanced datasets.
  - **Mean Absolute Error (MAE)**: Measures the average magnitude of errors in predictions, used for regression tasks.

In summary:
- **Optimizer**: Adjusts the weights to minimize the loss.
- **Loss Function**: Quantifies how far the model's predictions are from the actual values.
- **Metric**: Evaluates the model’s performance and helps in monitoring its progress.

## Compile the model

In [66]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.1)

In [69]:
# compile the model
# Optimizer: Updates weights during training. Common choices are 'adam', 'sgd', etc.
# Loss Function: Measures the error. For binary classification, use binary_crossentropy.
# Metrics: Metrics like accuracy are used to evaluate the model's performance.

model.compile(optimizer = opt, 
              loss="binary_crossentropy",
              metrics= ["accuracy"])

## Setup the Tensorboard and EarlyStopping

### **Early Stopping**

**Definition**: Early stopping is a regularization technique used during the training of neural networks to prevent overfitting. It monitors a specific metric, usually the validation loss or accuracy, and stops training when the metric stops improving.

**Purpose**: To avoid overfitting by halting training when the model's performance on the validation set starts to degrade, thereby saving computational resources and improving model generalization.

**How It Works**:
- During training, the algorithm keeps track of the performance metric (e.g., validation loss).
- If the metric does not improve for a specified number of epochs (patience), training is stopped early.
- This ensures that the model does not continue to train excessively and overfit the training data.

### **TensorBoard**

**Definition**: TensorBoard is a visualization tool for TensorFlow that provides a suite of visualizations to understand, debug, and optimize machine learning models.

**Purpose**: To track and visualize various metrics, such as training and validation loss, accuracy, and other custom metrics, as well as to inspect model architectures, histograms of weights, and more.

**Features**:
- **Scalars**: Track and visualize metrics like loss and accuracy over epochs.
- **Graphs**: Visualize the computation graph to understand the model architecture and data flow.
- **Histograms**: Inspect distributions of weights, biases, and other parameters.
- **Images**: Display image data and predictions.

**How It Works**:
- During training, TensorBoard logs metrics and other information into a log directory.
- The `TensorBoard` callback is used to write these logs, which can then be viewed in the TensorBoard web application.

**To View TensorBoard**:
1. Run the following command in the terminal to start TensorBoard:
   ```bash
   tensorboard --logdir=logs/fit
   ```
2. Open your browser and go to `http://localhost:6006` to view the TensorBoard dashboard.

In summary:
- **Early Stopping**: Stops training early when the monitored metric ceases to improve, helping to prevent overfitting.
- **TensorBoard**: Provides a suite of visualization tools to monitor and analyze the training process of machine learning models.

In [76]:
# Setup the Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [78]:
# Setup Early Stopping
early_stopping_callback = EarlyStopping(monitor = "val_loss",
                                        patience = 5,
                                        restore_best_weights=True)

## Training the Model

In [79]:
history = model.fit(
    X_train, y_train, validation_data= (X_test,y_test), epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 0.4547 - accuracy: 0.8191 - val_loss: 0.4708 - val_accuracy: 0.8155
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4163 - accuracy: 0.8401 - val_loss: 0.3789 - val_accuracy: 0.8570
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4551 - accuracy: 0.8307 - val_loss: 0.4320 - val_accuracy: 0.8535
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4329 - accuracy: 0.8414 - val_loss: 0.3801 - val_accuracy: 0.8555
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4264 - accuracy: 0.8401 - val_loss: 0.4097 - val_accuracy: 0.8570
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4260 - accuracy: 0.8400 - val_loss: 0.4030 - val_accuracy: 0.8330
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4134 - accuracy: 0.8430 - val_loss: 0.3747 - val_accuracy: 0.8540

## Saving the model

In [80]:
# Saving model

model.save("model.h5")

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Currently tensorboard is not working

In [81]:
# Load TensorBoard
%load_ext tensorboard

In [84]:
#%tensorboard --logdir /kaggle/working/logs/fit20240816-231541

# Prediction Pipeline: Data Loading, Preprocessing, and Prediction

In [85]:
# Predication
# Load all files
from tensorflow.keras.models import load_model
model = load_model("/kaggle/working/model.h5")

# Load encoder and scaler
with open("/kaggle/working/ohe_geo.pkl", "rb") as file:
    label_encoder_geo_file = pickle.load(file)
    
with open("/kaggle/working/label_encoder_gender.pkl", "rb") as file:
    label_encoder_gender_file = pickle.load(file)

with open("/kaggle/working/scaler.pkl", "rb") as file:
    scaler_file = pickle.load(file)

In [106]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [107]:
# One hot encode "Geography"
geo_encode = label_encoder_geo_file.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encode, columns=label_encoder_geo_file.get_feature_names_out(["Geography"]))
geo_encoded_df

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [109]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [110]:
# Encode the categorical column

input_df['Gender']=label_encoder_gender_file.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [111]:
# Concat OHE data
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis = 1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [112]:
# Scaling the data
input_scaled= scaler_file.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [113]:
# Prediction
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 107ms/step


array([[0.09841255]], dtype=float32)

In [116]:
prediction_prob = prediction[0][0]
prediction_prob

0.09841255

In [117]:
if prediction_prob > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
